In [9]:
import sys
sys.path.append('..')
from db1 import *
import pandas as pd

R = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * kazhdan_lusztig_uniform_matroid(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_uniform_matroid(k, n) + kazhdan_lusztig_uniform_matroid(k-c+1, h) * kazhdan_lusztig_uniform_matroid(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [10]:
def parallel_connection(m, n):
    G = graphs.CycleGraph(m + n - 2)
    G.add_edge(0, m-1)
    return G

In [11]:
for n in range (3, 15):
    G = graphs.CycleGraph(n)
    print(kl_inverse_fast(Matroid(G)))

2
2*t + 3
5*t + 4
5*t^2 + 9*t + 5
14*t^2 + 14*t + 6
14*t^3 + 28*t^2 + 20*t + 7
42*t^3 + 48*t^2 + 27*t + 8
42*t^4 + 90*t^3 + 75*t^2 + 35*t + 9
132*t^4 + 165*t^3 + 110*t^2 + 44*t + 10
132*t^5 + 297*t^4 + 275*t^3 + 154*t^2 + 54*t + 11
429*t^5 + 572*t^4 + 429*t^3 + 208*t^2 + 65*t + 12
429*t^6 + 1001*t^5 + 1001*t^4 + 637*t^3 + 273*t^2 + 77*t + 13


In [12]:
for n in range (2, 15):
    M = matroids.Uniform(n, 15)
    print(kl_inverse_fast(M))

14
90*t + 91
715*t + 364
1925*t^2 + 2925*t + 1001
9450*t^2 + 7722*t + 2002
13650*t^3 + 24948*t^2 + 14300*t + 3003
42042*t^3 + 43120*t^2 + 19305*t + 3432
34398*t^4 + 70070*t^3 + 51975*t^2 + 19305*t + 3003
63700*t^4 + 75075*t^3 + 44550*t^2 + 14300*t + 2002
28028*t^5 + 61425*t^4 + 53625*t^3 + 26950*t^2 + 7722*t + 1001
27027*t^5 + 35100*t^4 + 25025*t^3 + 11088*t^2 + 2925*t + 364
5005*t^6 + 11583*t^5 + 11375*t^4 + 7007*t^3 + 2835*t^2 + 715*t + 91
1430*t^6 + 2002*t^5 + 1638*t^4 + 910*t^3 + 350*t^2 + 90*t + 14


In [13]:
for n in range (2, 15):
    M = Matroid(parallel_connection(n, n))
    print(kl_inverse_fast(M))

1
t + 4
5*t^2 + 13*t + 9
10*t^3 + 32*t^2 + 41*t + 16
42*t^4 + 115*t^3 + 140*t^2 + 91*t + 25
107*t^5 + 322*t^4 + 446*t^3 + 375*t^2 + 169*t + 36
429*t^6 + 1197*t^5 + 1589*t^4 + 1393*t^3 + 805*t^2 + 281*t + 49
1234*t^7 + 3628*t^6 + 5208*t^5 + 5012*t^4 + 3368*t^3 + 1512*t^2 + 433*t + 64
4862*t^8 + 13698*t^7 + 19056*t^6 + 18600*t^5 + 13344*t^4 + 7005*t^3 + 2592*t^2 + 631*t + 81
15032*t^9 + 43754*t^8 + 64110*t^7 + 66660*t^6 + 51621*t^5 + 30210*t^4 + 13150*t^3 + 4155*t^2 + 881*t + 100
58786*t^10 + 166650*t^9 + 238634*t^8 + 249051*t^7 + 200112*t^6 + 125455*t^5 + 61259*t^4 + 22891*t^3 + 6325*t^2 + 1189*t + 121
190588*t^11 + 552312*t^10 + 820369*t^9 + 896192*t^8 + 760474*t^7 + 509641*t^6 + 271722*t^5 + 114510*t^4 + 37588*t^3 + 9240*t^2 + 1561*t + 144
742900*t^12 + 2115581*t^11 + 3089242*t^10 + 3373227*t^9 + 2926781*t^8 + 2051478*t^7 + 1168453*t^6 + 540332*t^5 + 200928*t^4 + 58903*t^3 + 13052*t^2 + 2003*t + 169


In [15]:
for n in range (3, 10):
    M = matroids.Wheel(n)
    print(kl_inverse_fast(M))

t + 6
9*t + 14
5*t^2 + 31*t + 30
36*t^2 + 85*t + 62
21*t^3 + 133*t^2 + 211*t + 126
140*t^3 + 400*t^2 + 497*t + 254
84*t^4 + 540*t^3 + 1089*t^2 + 1135*t + 510
